In [28]:
#Use PySpark to initialize the SparkContext.   
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [29]:
# Creating a SparkContext object  
sc = SparkContext.getOrCreate()

# Creating a SparkSession  
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Extract employee data from CSV, and transform using filtering, grouping, joins, and aggregations as a new DataFrame


#### Extract


In [49]:
# Define a Schema for the input data and read the file using the user-defined Schema
emp_schema = employees_df.schema
employees_df = spark.read.load(csv_file, schema=emp_schema, format='csv', header=True)

In [47]:
# Display all columns of the DataFrame, along with their respective data types
employees_df.printSchema()

root
 |-- Emp_No: string (nullable = true)
 |-- Emp_Name: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Department: string (nullable = true)



In [48]:
# SQL query to fetch solely the records from the View where the age exceeds 30
result = spark.sql("SELECT * FROM employees WHERE age > 30")
result.show()

+------+-----------+------+---+----------+
|Emp_No|   Emp_Name|Salary|Age|Department|
+------+-----------+------+---+----------+
|   199|    Douglas|  2600| 34|     Sales|
|   200|   Jennifer|  4400| 36| Marketing|
|   201|    Michael| 13000| 32|        IT|
|   202|        Pat|  6000| 39|        HR|
|   203|      Susan|  6500| 36| Marketing|
|   205|    Shelley| 12008| 33|   Finance|
|   206|    William|  8300| 37|        IT|
|   100|     Steven| 24000| 39|        IT|
|   102|        Lex| 17000| 37| Marketing|
|   103|  Alexander|  9000| 39| Marketing|
|   104|      Bruce|  6000| 38|        IT|
|   105|      David|  4800| 39|        IT|
|   106|      Valli|  4800| 38|     Sales|
|   107|      Diana|  4200| 35|     Sales|
|   109|     Daniel|  9000| 35|        HR|
|   110|       John|  8200| 31| Marketing|
|   111|     Ismael|  7700| 32|        IT|
|   112|Jose Manuel|  7800| 34|        HR|
|   113|       Luis|  6900| 34|     Sales|
|   116|     Shelli|  2900| 37|   Finance|
+------+---

#### Transform

In [ ]:
# SQL query to calculate the average salary of employees grouped by department
result = spark.sql('SELECT Department, avg(Salary) as Average_salary FROM employees group by Department')
result.show()

In [ ]:
# Apply a filter to select records where the department is 'IT'
result = employees_df.filter(employees_df.Department == 'IT')
result.show()

In [ ]:
from pyspark.sql.functions import col

# Add a new column "SalaryAfterBonus" with 10% bonus added to the original salary
employees_df = employees_df.withColumn('SalaryAfterBonus', employees_df.Salary*1.1)

In [ ]:
from pyspark.sql.functions import max

# Group data by age and calculate the maximum salary for each age group
employees_df.groupBy('Age').agg(max('Salary')).show()

In [ ]:
# Join the DataFrame with itself based on the "Emp_No" column
employees_df.join(employees_df, 'Emp_No', 'inner').show()

In [ ]:
# Calculate the average age of employees
from pyspark.sql.functions import avg 

employees_df.agg(avg('Age')).show()

In [34]:
# Calculate the total salary for each department.
from pyspark.sql.functions import sum 

employees_df.groupBy('Department').agg(sum('Salary')).show()

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|      71408|
|        HR|      46700|
|   Finance|      57308|
| Marketing|      59700|
|        IT|      74000|
+----------+-----------+



In [35]:
# Sort the DataFrame by age in ascending order and then by salary in descending order
from pyspark.sql.functions import asc, desc

employees_df.sort(asc('Age'))
employees_df.sort(desc('Salary'))

employees_df.show()

+------+---------+------+---+----------+
|Emp_No| Emp_Name|Salary|Age|Department|
+------+---------+------+---+----------+
|   198|   Donald|  2600| 29|        IT|
|   199|  Douglas|  2600| 34|     Sales|
|   200| Jennifer|  4400| 36| Marketing|
|   201|  Michael| 13000| 32|        IT|
|   202|      Pat|  6000| 39|        HR|
|   203|    Susan|  6500| 36| Marketing|
|   204|  Hermann| 10000| 29|   Finance|
|   205|  Shelley| 12008| 33|   Finance|
|   206|  William|  8300| 37|        IT|
|   100|   Steven| 24000| 39|        IT|
|   101|    Neena| 17000| 27|     Sales|
|   102|      Lex| 17000| 37| Marketing|
|   103|Alexander|  9000| 39| Marketing|
|   104|    Bruce|  6000| 38|        IT|
|   105|    David|  4800| 39|        IT|
|   106|    Valli|  4800| 38|     Sales|
|   107|    Diana|  4200| 35|     Sales|
|   108|    Nancy| 12008| 28|     Sales|
|   109|   Daniel|  9000| 35|        HR|
|   110|     John|  8200| 31| Marketing|
+------+---------+------+---+----------+
only showing top

In [39]:
from pyspark.sql.functions import count

# Calculate the number of employees in each department
employees_df.groupBy('Department').agg(count('Emp_No')).show()

+----------+-------------+
|Department|count(Emp_No)|
+----------+-------------+
|     Sales|           13|
|        HR|            8|
|   Finance|           10|
| Marketing|            9|
|        IT|           10|
+----------+-------------+



In [42]:
# Apply a filter to select records where the employee's name contains the letter 'o'
employees_df.filter(employees_df.Emp_Name.like('%o%')).show()

+------+-----------+------+---+----------+
|Emp_No|   Emp_Name|Salary|Age|Department|
+------+-----------+------+---+----------+
|   198|     Donald|  2600| 29|        IT|
|   199|    Douglas|  2600| 34|     Sales|
|   110|       John|  8200| 31| Marketing|
|   112|Jose Manuel|  7800| 34|        HR|
|   130|      Mozhe|  2800| 28| Marketing|
|   133|      Jason|  3300| 38|     Sales|
|   139|       John|  2700| 36|     Sales|
|   140|     Joshua|  2500| 29|   Finance|
+------+-----------+------+---+----------+

